In [1]:
import json

with open('nouns_adj.json', 'r') as f:
    documents = json.load(f)

In [2]:
import numpy as np
from gensim import matutils

def bow_iterator(docs, dictionary):
    for doc in docs:
        yield dictionary.doc2bow(doc)

def get_term_matrix(msgs, dictionary):
    bow = bow_iterator(msgs, dictionary)
    X = np.transpose(matutils.corpus2csc(bow).astype(np.int64))
    return X

### Filter out tokens by frequency

In [3]:
from gensim import corpora

# Map each token to a unique ID
dictionary = corpora.Dictionary(documents)
print(f'Number of unique tokens: {len(dictionary)}')

# Filter out words that occur in more than 30% of the documents
min_doc, max_doc = 15, .3
dictionary.filter_extremes(no_below=min_doc, no_above=max_doc)
print(f'Number of unique tokens (filtered): {len(dictionary)}')

Number of unique tokens: 30281
Number of unique tokens (filtered): 3346


In [4]:
# Store dictionary items as words in vocab
tmp = []
for i in range(len(dictionary)):
    tmp.append(dictionary.get(i))
vocab = tuple(tmp)

word2id = dict((v, idx) for idx, v in enumerate(vocab))

In [5]:
# Create a sparse matrix from gensim's bow document representation
X = get_term_matrix(documents, dictionary)

### Build a Guided LDA model w/o seed topics

In [6]:
from lda import guidedlda as glda

n_topics = 15

# Train a Guided LDA model
glda_model = glda.GuidedLDA(alpha=.1, n_topics=n_topics, n_iter=300, \
                            random_state=7, refresh=20)

# Run the model
glda_model.fit(X, seed_confidence=0.6)

INFO:lda:n_documents: 1260
INFO:lda:vocab_size: 3346
INFO:lda:n_words: 409615
INFO:lda:n_topics: 15
INFO:lda:n_iter: 300
INFO:lda:<0> log likelihood: -4448624
INFO:lda:<20> log likelihood: -3104520
INFO:lda:<40> log likelihood: -3055555
INFO:lda:<60> log likelihood: -3038425
INFO:lda:<80> log likelihood: -3028541
INFO:lda:<100> log likelihood: -3023128
INFO:lda:<120> log likelihood: -3017305
INFO:lda:<140> log likelihood: -3013047
INFO:lda:<160> log likelihood: -3011456
INFO:lda:<180> log likelihood: -3009448
INFO:lda:<200> log likelihood: -3007417
INFO:lda:<220> log likelihood: -3008095
INFO:lda:<240> log likelihood: -3006962
INFO:lda:<260> log likelihood: -3006861
INFO:lda:<280> log likelihood: -3006296
INFO:lda:<299> log likelihood: -3007284


### Print the results

In [7]:
n_top_words = 15
topic_word = glda_model.topic_word_

for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print(f"Topic {i}: {' '.join(topic_words)}\n")

Topic 0: электрон атом поле температура магнитный_поле квантовый энергия частота сверхпроводник сверхпроводимость сила магнитный величина излучение взаимодействие

Topic 1: самец самка особь поведение популяция отбор половой окраска признак потомство яйцо колония муха генетический пара

Topic 2: мозг нейрон мышь сигнал рецептор поведение запах реакция действие звук тело память млекопитающее частота культура

Topic 3: теория научный наука проект решение открытие задача журнал история полный профессор коллега публикация слово век

Topic 4: молекула химический реакция соединение материал водород свойство синтез бактерия вода атом температура графен молекулярный ион

Topic 5: ген белок мутация днк геном опухоль экспрессия ткань мышь участок рак вирус бактерия белка фермент

Topic 6: порода зона состав образование слой мантия железо глубина период возраст минерал планета материал граница древний

Topic 7: свет объект изображение направление длина внутренний расстояние отдельный глаз информа

### Build a model with seed topics

In [10]:
seed_topic_list = [['сапиенс', 'пещера', 'неандерталец', 'находка', 'кость'],
                   ['галактика', 'чёрный_дыра', 'вселенная', 'космический'],
                   ['опухоль', 'рак', 'метастаз', 'раковый', 'мутация'],
                   ['коллайдер', 'распад', 'детектор', 'бозон_хиггс', 'кварк'],
                   ['атмосфера', 'содержание', 'климат', 'концентрация'],
                   ['нейрон', 'мозг', 'рецептор', 'сигнал', 'нервный']]

In [11]:
seed_topics = {}

for t_id, st in enumerate(seed_topic_list):
    for word in st:
        seed_topics[word2id[word]] = t_id

# Run the model with seed topics
glda_model.fit(X, seed_topics=seed_topics, seed_confidence=0.15)

INFO:lda:n_documents: 1260
INFO:lda:vocab_size: 3346
INFO:lda:n_words: 409615
INFO:lda:n_topics: 15
INFO:lda:n_iter: 300
INFO:lda:<0> log likelihood: -4447106
INFO:lda:<20> log likelihood: -3095018
INFO:lda:<40> log likelihood: -3049578
INFO:lda:<60> log likelihood: -3036288
INFO:lda:<80> log likelihood: -3030521
INFO:lda:<100> log likelihood: -3025782
INFO:lda:<120> log likelihood: -3022618
INFO:lda:<140> log likelihood: -3021741
INFO:lda:<160> log likelihood: -3020174
INFO:lda:<180> log likelihood: -3018076
INFO:lda:<200> log likelihood: -3016947
INFO:lda:<220> log likelihood: -3015505
INFO:lda:<240> log likelihood: -3015523
INFO:lda:<260> log likelihood: -3015176
INFO:lda:<280> log likelihood: -3013404
INFO:lda:<299> log likelihood: -3013434


### Print the results

In [12]:
topic_word = glda_model.topic_word_
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print(f"Topic {i}: {' '.join(topic_words)}\n")

Topic 0: древний возраст кость геном слой находка поздний след популяция днк сапиенс неандерталец зуб признак генетический

Topic 1: звезда масса галактика чёрный_дыра вселенная излучение солнечный космический наблюдение астроном солнце звёздный спектр ядро дыра

Topic 2: белок ген днк опухоль мутация мышь ткань рак участок вирус бактерия геном фермент белка рецептор

Topic 3: частица энергия физика ядро электрон нейтрино детектор протон измерение масса распад коллаборация событие коллайдер нейтрон

Topic 4: вода температура атмосфера океан период порода содержание слой кислород глубина морской углерод состав зона лёд

Topic 5: самец самка поведение мозг нейрон сигнал особь половой муха запах потомство отбор частота популяция различие

Topic 6: сверхпроводник сверхпроводимость магнитный_поле мантия температура сверхпроводящий магнитный ядро зона материал вихрь образование критический_температура плёнка поле

Topic 7: молекула химический реакция вода атом водород синтез свойство соедине